In [4]:
import tarfile
import os

# Define the path to your model directory and the output archive file
model_directory = 'Model/fine_tuned_bert_model.pt'  # Path to your model directory
output_archive = 'newModel'  # Desired path for the output .tar.gz file


In [10]:
pip install boto3 transformers


Note: you may need to restart the kernel to use updated packages.


In [5]:
# Creating the .tar.gz archive
with tarfile.open(output_archive, "w:gz") as tar_handle:
    for root, dirs, files in os.walk(model_directory):
        for file in files:
            tar_handle.add(os.path.join(root, file), arcname=file)

print(f"Packaged model and inference script into {output_archive}")

Packaged model and inference script into newModel


In [6]:
import boto3

s3_resource = boto3.resource('s3')
bucket_name = 'aihackathon149' 
s3_key = 'models/2024-03-07/aihackathon-model.tar.gz' 

# Upload the .tar.gz file to S3
s3_resource.Bucket(bucket_name).upload_file(Filename=output_archive, Key=s3_key)

print(f"Uploaded the model to s3://{bucket_name}/{s3_key}")

Uploaded the model to s3://aihackathon149/models/2024-03-07/aihackathon-model.tar.gz


In [7]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

role = get_execution_role()  # SageMaker execution role
model_data = f's3://{bucket_name}/{s3_key}'  # S3 path to your packaged model



In [8]:
# Define the PyTorch model object
print("Hello")
pytorch_model = PyTorchModel(model_data=model_data,
                             role=role,
                             entry_point='Model/inference.py',  # Your inference script
                             framework_version='1.7.1',  # Match the version used for training
                             py_version='py3')
                        
# Deploy the model to an endpoint
predictor = pytorch_model.deploy(instance_type='ml.m5.large', initial_instance_count=1)
print("Hello 3")
print(f"Model deployed at endpoint: {predictor.endpoint_name}")
print("Hello 4")

Hello
----!Hello 3
Model deployed at endpoint: pytorch-inference-2024-03-07-07-22-10-795
Hello 4


In [ ]:
# Example test data
test_data = 'Riyadh to Makka'

# Get predictions
response = predictor.predict(test_data)
print(response)